In [ ]:
!pip uninstall -y sentence-transformers
!pip install torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install -q "transformers==4.43.2" "datasets==2.18.0" "accelerate==0.29.3" "peft==0.10.0" "bitsandbytes==0.43.1" "trl==0.8.6" "protobuf==3.20.3"
!pip install -q einops scipy sentencepiece tensorboard

import os
os.kill(os.getpid(), 9)

Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 119.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 130.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4

In [ ]:
from google.colab import userdata, files
from huggingface_hub import login
from datasets import load_dataset
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


In [ ]:
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    login(token=HF_TOKEN)
    print("Successfully logged into Hugging Face.")
except Exception as e:
    print(f"Login failed. Please set your HF_TOKEN in Colab Secrets. Error: {e}")
    raise SystemExit("Stopping.")

print("\nPlease upload your 'training_data_llm_generated_final.jsonl' file.")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Stopping.")
    raise SystemExit("Stopping.")

DATASET_NAME = list(uploaded.keys())[0]
dataset = load_dataset('json', data_files=DATASET_NAME, split='train')
print("\nDataset loaded successfully.")
print(dataset)

✅ Successfully logged into Hugging Face.

Please upload your 'training_data_llm_generated_final.jsonl' file.


Saving training_data_llm_generated_final.jsonl to training_data_llm_generated_final (2).jsonl


Generating train split: 0 examples [00:00, ? examples/s]


✅ Dataset loaded successfully.
Dataset({
    features: ['input', 'output'],
    num_rows: 600
})


In [ ]:
def format_example(example):
    instruction = "Analyze the provided patient narrative about their experience with an Abdominal Wall Hernia (AWH) and generate a structured JSON output that summarizes your findings, adhering to the specified format and terminology."
    input_text = example['input']
    output_text = example['output']
    return {'text': f"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{instruction}\n\n**Patient Narrative:**\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n{output_text}<|eot_id|>"}

formatted_dataset = dataset.map(format_example)
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, max_length=8192)

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True, remove_columns=['input', 'output', 'text'], num_proc=2)
print("Dataset formatted and tokenized for a full 8k context window.")


bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
print("Model configured with a powerful QLoRA (r=32).")

training_args = TrainingArguments(
    output_dir='./llama3-hernia-analyst-A100-v1',
    num_train_epochs=1,
    per_device_train_batch_size=2, # Using a safe but effective batch size for 8k sequences
    gradient_accumulation_steps=4, # Effective batch size = 2 * 4 = 8
    learning_rate=2e-4,
    bf16=True,
    save_strategy="epoch",
    logging_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    report_to="none"
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset, tokenizer=tokenizer, data_collator=data_collator)

print("\nStarting the A100-optimized fine-tuning process...")
trainer.train()
print("Training complete!")

trainer.save_model('./llama3-hernia-analyst-A100-v1')
print("High-quality model adapter saved successfully.")

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/600 [00:00<?, ? examples/s]

✅ Dataset formatted and tokenized for a full 8k context window.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model configured with a powerful QLoRA (r=32).

🚀 Starting the A100-optimized fine-tuning process...


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.333100
20,0.931500
30,0.871900
40,0.824900
50,0.803800
60,0.777300
70,0.775000


✅ Training complete!
✅ High-quality model adapter saved successfully.
